# Baiscs of mobile robotics Project

Group n°32
Students: 
- Konstantin Dinev (314419)
- Owen Timothy Claude Simon (330807)
- Jiarui Yu (351440)
- Jeffrey Siyuan Yu (371327)

## Introduction

The story centers on Little Thymio, who is assigned to deliver vital information to the main station while stationed on the moon. Little Thymio uses telescopes to map the lunar surface and navigate, but he has a difficulty when attempting to discover the shortest path because the background is subject to abrupt changes. Thymio uses local obstacle avoidance, deftly avoiding preset black polygon impediments, to guarantee a safe voyage.


Complicating matters, certain areas of the moon have bad ground station connectivity, which interferes with communication and picture delivery. During communication failures, Little Thymio cleverly uses a Kalman filter and its inbuilt sensors to survive until reconnection. In this lunar journey, the objective is a calm blue circle known as the moon station, and Thymio, as seen through telescopes, is symbolized by a unique red triangle. The setting is show in the figure below.

<div align=center>
<img src="./figs/setup.jpg"/>
<figcaption>Thymio, obstacle and marker setup</figcaption>
</div>

The real environment setting is shown as below 

<div align=center>
<img src="./figs/real_env.jpg" alt="Real implement environment" style="width:40%"/> 
<figcaption>Real implement environment</figcaption>
</div>


## Convention settings

## Vision

The image processing steps are aimed at identifying and understanding different elements in a scene. Initially, the focus is on detecting green squares, which serve as reference points for transforming the entire image. The process then involves recognizing obstacles in black, identifying the goal in blue, and pinpointing the robot through a distinctive red triangle.
Originally, we attempted to use QR code detection for localization, but it proved less reliable. Subsequently, we turned to Aruco markers, which worked well for a smaller robot map. However, as the map size increased, the camera struggled to consistently detect QR codes in each frame. This challenge led us to adopt a simpler approach, using colored triangles to represent the robot's location.

After switching the Thymio to a triangle, we still had issues with detection. The camera only detected the Thymio triangle approximately one out of 30 loops. To resolve this, we plotted the original image and the masks. Immediately, we noticed the first frame of the video captured always had a blue hue, but the mask clearly detected the triangle. From the second capture onwards, the camera lost the hue and the mask could not detect any triangle. We confirmed this behavior by running the script multiple times. To resolve this issue, we added a statement to ignore the first 50 frames from the webcam upon startup. This solved the issue with the Thymio detection.

## Processing the obstacles

After detecting the obstacles, their boundaries need to be expanded to account for the size of the Thymio. This is because the global path planning algorithm uses the vertices of the obstacles and the center of the Thymio to create the path. Consequently, the distance from the Thymio's center to its farthest edge must be added to every point on the obstacle prevent collisions.

The most effective approach is to depict the Thymio as a circumscribing circle, ensuring that its orientation doesn't impact the necessary offset size. By moving this circle along the obstacle's perimeter while it tangentially touches each edge, we achieve the desired offset for expanding the obstacle shape.  However, this is a complex operation to perform within openCV, so we used a simpler solution that creates an offset approximation by scaling the contours.

The function takes two inputs, the contour of each obstacle and the desired_min_distance as inputs. The desired_min_distance is equivalent to the radius of the circumscribing circle around the Thymio. It and scales the countours so that each pixel on the new contour is at least desired_min_distance away from the closest pixel in the old contour. If not, the scale factor is increased, the new contour is calculated, and checked. This process iterates until all points on the new contour satisfy the constraint. 

This rough approximation works best on shapes that are symmetric. This is because the scaling of is performed uniformly in the x and y directions. In our case, a rectangle scales symmetrically so it was chosen as an obstacle. (IMAGE BELOW TO SHOW THE EXPANDED OBSTACLE)

In [ ]:
# INPUTS: a contour, and minimum distance in pixels needed to scale the contour
def scale_contour(original_contour, desired_min_distance):
    # Get the bounding rectangle around the shape
    x, y, w, h = cv2.boundingRect(original_contour)

    # Calculate the center of the bounding rectangle
    center = ((x + w // 2), (y + h // 2))

    scaled_adequate = False
    scale_factor = 1.3

    while (not scaled_adequate):
        # Scale each point of the contour relative to the center
        scaled_contour = np.array([[(point[0][0] - center[0]) * scale_factor + center[0],
                                    (point[0][1] - center[1]) * scale_factor + center[1]]
                                   for point in original_contour], dtype=np.int32)
        
        min_distance = float('inf')
        # checking if every point in the contour is at least desired_min_distance 
        # away from the closest point in the orignal contour
        for point in scaled_contour:
            point = tuple(float(coord) for coord in point)
            distance = cv2.pointPolygonTest(original_contour, point, True)
            min_distance = min(min_distance, abs(distance))
        # increase scaling factor if the minimal distance is not achieved
        if (min_distance < desired_min_distance):
            scale_factor += 0.01

        else:
            scaled_adequate = True
            print("Adequate Scaling achieved for obstacles")

    return scaled_contour

## Global navigation

Global navigation is used to create the shortest path to the destination. The solution two-fold, first create a visibility graph with the current location of the thymio, the expanded vertices of the obstacles, and the center of the goal as nodes. A node is connected to another if there is a direct path between the two. After creating the adjacency graph, a shortest path algorithm can be used, A-star or Dijkstra for example. In this application, after we understood a method to find the shortest path, we used a library PyVisGraph to help with implementation of creating a visibility graph and finding the shortest path. This library employs Dijkstra's algorithm. While we acknowledge that A-star is more computationally efficient, for the current application, Dijkstra's algorithm demonstrated satisfactory speed performance within the computational capabilities of a laptop. (INSERT IMAGE OF GLOBAL PATH DRAWN)

In [ ]:
# INPUTS: vertices of obstacles, the robot position, goal position
def get_shortest_path(shape_vertices, rob_pos, goal_pos):
    polygons = []
    for shape in shape_vertices:
        polygon = []
        for point in shape:
            polygon.append(vg.Point(point[0], point[1]))
        polygons.append(polygon)

    # creating the visibility graph
    graph = vg.VisGraph()
    graph.build(polygons)

    startPosition = vg.Point(rob_pos[0], rob_pos[1])
    endPosition = vg.Point(goal_pos[0], goal_pos[1])

    shortestPath = graph.shortest_path(startPosition, endPosition)
    return shortestPath

## Thymio Class and Motion Control

## Kalman filter

### Why do we need kalman filter?

In this project, the thymio robot is controlled based on the measurements of a webcam and a color, simple code based traingle detection for positioning, therefore, the robot's position detection can be inaccurate and not robust. Under this circumstance, integrating a Kalman filter can be of great benefit. To produces a more accurate estimation of the robot's orientation and position, the filter fuses the predicted outcomes based on the robot's dynamics defined in the filter and the wheel speed encoders with the measurements from the webcam. Moreover, the Kalman filter's ability to predict future states using its own dynamic models allows it to make predictions even without the webcam measurements. In this way, the robot can have a basic knowledge of its position and orientation when the positioning from the camera fails or the camera fails.


### Why we use Extended kalman filter?

Conventional kalman filter can deal with linear and discrete time dyanmics, however, the thymio dynamic is non-linear, therefore, inspired by both non-linear hearistic and lectures about kalman in Multivariable course in EPFL, Extened Kalman Filter (EKF) which is the non-linear version of Kalman filter seems to be a better choice to be implemented in this project. EKF linearizes the system around the current estimation mean and covariance, giving a reasonable performance and being the de facto standard in navigation systems although it is not optimal.

To have a basic knowledge of how to implement the EKF,  we ask chatGPT to give us a basic structure of the EKF. However, its answer may be designed for some more complex situations where we cannot express dynamics in matrices but equations instead. So we change the structure of the EKF with the help of https://github.com/mez/extended_kalman_filter_python/blob/master/ekf.py.

### Instruction of dynamics

Although the thymio system is continuous, we control it in a discrete time way in every control loop. As the postion of  thymio is consisted of its location and orientation, also the movement of the robot is controlled by its two wheels' speeds, for each time step t or loop t,we have introduced the state vector $s_{state}$ to be
$$
s_{state_t} 
= 
\begin{gather}
 \begin{bmatrix}
           x_{t} \\
           y_{t} \\
           \theta_{t} \\
           v_{lw_{t}} \\
           v_{rw_{t}} 
 \end{bmatrix}\nonumber
\end{gather} 


$$
where x denotes the x-axis position, y denotes the y-axis position, $\theta$ denotes the thymio orientation according to the convention setting, $v_{lw}$ denotes the left wheel velocity and $v_{rw}$ denotes the right wheel velocity.  Generally, the dynamic can be formulated as below.

$$
 \begin{gather}
  \begin{bmatrix}
        x_{t+1} \\
        y_{t+1} \\
        \theta_{t+1}\\
        v_{lw_{t+1}} \\
        v_{rw_{t+1}} 
   \end{bmatrix} \nonumber
\end{gather}
 =
 \begin{gather}
  \begin{bmatrix}
   x_{t} + dT*cos(\theta_{t})*v_{forward_t} \\[10pt]
   y_{t} + dT*sin(\theta_{t})*v_{forward_t} \\[10pt]
   \theta_{t} + dT* v_{turn}\\[10pt]
           v_{lw_{t}} \\[10pt]
           v_{rw_{t}} 
   \end{bmatrix} \nonumber
\end{gather}
$$
As the thymio moving vector is only influenced by its two wheels' speed, we could have the following equations 
$$
v_{forward_t} = \dfrac{v_{lw_{t}}+v_{rw_{t}}}{2} \nonumber \\
v_{turn_t} = \dfrac{v_{lw_{t}}-v_{rw_{t}}}{l_{wheels}}\nonumber
$$
where $dT$ denotes the time for each control loop (sampling period for thymio motor) and $l_{wheels}$ denotes the distance between thymio wheels. We then introduce matrix f and mateix h in the EKF to be 
$$ 
\mathbf{f}
=
\begin{bmatrix}
1 & 0 & 0 & \dfrac{dT*cos(\theta_{t-1})}{2} & \dfrac{dT*cos(\theta_{t-1})}{2} \\[10pt]
0 & 1 & 0 & \dfrac{dT*sin(\theta_{k-1})}{2} & \dfrac{dT*sin(\theta_{k-1})}{2} \\[10pt]
0 & 0 & 1 & \dfrac{dT}{l_{wheels}} & -\dfrac{dT}{l_{wheels}} \\[10pt]
0 & 0 & 0 & 1 & 0 \\[10pt]
0 & 0 & 0 & 0 & 1 
\end{bmatrix}
$$
$$ 
\mathbf{h}
=
diag([1, 1, 1, 1, 1])
$$


With matrix f and h, the dynamics in the EKF can be defined as 
$$ s_{state_{t+1}} = f*s_{state_{t}}+w$$
$$y_t=h*s_{state_t}+v$$

where w and v denote the noises  on  process and observation respectively.


### Instruction of EKF 


In EKF, a jaccobian matrix of the dynamic is required for the update of covariance terms. As we aleardy have the derivatives of states as matrix f and h, we could simple take another derivative over states of matrix f and h. We use matrix F and H to denote the Jacobian of process and observation respectively and they are shown as below.

$$
\mathbf{F}
=
\begin{bmatrix}
1 & 0 & v_{forward_t}*dT*sin(\theta_{t}) & \dfrac{dT*cos(\theta_{t})}{2} & \dfrac{dT*cos(\theta_{t})}{2} \\[10pt]
0 & 1 & v_{forward_t}*dT*cos(\theta_{t}) & \dfrac{dT*sin(\theta_{t})}{2} & \dfrac{dT*sin(\theta_{t})}{2} \\[10pt]
0 & 0 & 1 & \dfrac{dT}{l_{wheels}} & -\dfrac{dT}{l_{wheels}} \\[10pt]
0 & 0 & 0 & 1 & 0 \\[10pt]
0 & 0 & 0 & 0 & 1 [10pt]
\end{bmatrix}
$$
$$ 
\mathbf{H}
= diag([1, 1, 1, 1, 1])
$$

The predict step equations and update step equations can be found on Wikipedia (https://en.wikipedia.org/wiki/Extended_Kalman_filter) and are shown as below:

\begin{align}

&\mathbf{Predict} \nonumber\\
&Predicted-state-estimate \quad \hat{x}_{k|k-1} = f*\hat{x}_{k-1|k-1} \nonumber\\
&Predicted-covariance-estimate \quad P_{k|k-1} = F_kP_{k-1|k-1}F_k^T + Q_{k-1}\nonumber\\
\nonumber\\
& \mathbf{Update} \nonumber\\
& Innovation \quad \tilde{y}_k=z_k-h(\hat{x}_{k-1|k-1}) \nonumber\\
& Innovation-covariance \quad S_k = H_kP_{k|k-1} H_k^T+R_k \nonumber\\
& Kalman-gain \quad K_k = P_{k|k-1}H_k^T{S_k}^{-1} \nonumber\\
& Updated-state-estimate \quad \hat{x}_{k|k} = \hat{x}_{k|k-1}+K_k\tilde{y}_k \nonumber\\
& Updated-covariance-estimate \quad P_{k|k}=(I-K_kH_k)P_{k|k-1} \nonumber

\end{align}

To be noticed, inspired by the guiding code mention above, we have implement the function that the thymio robot even work without webcam measuremnets by simply using the predictions of EFK dynamics. In this situation, as we are only using the wheel encoders, the matrix H has to be changed into following forms.

$$ 
\mathbf{H_{encoder}}
=
\begin{bmatrix}
0 & 0 \\
0 & 0 \\
0 & 0 \\
1 & 0 \\
0 & 1
\end{bmatrix}
$$


### Into the codes

#### Parameter settings
Besides the dynamics, the most import part in the EKF is the noise variance matrix. Firstly, we want to be as realistic as possible and therefore measures the varaince of the wheel speeds. The outcome we obtained is shown in the figure below. From the measurements, we have decided that the varaince of the wheel speed is $12.31 m^2/s$. However, we later found out that it is difficult to obtain the accurate webcam resolution especially when the webcam is not put in the same postion each time we set up the experiment environment (different height, different map size, different light condition...). So at last we decide to set these noise variances by try and trial. In detail, the speed varaince (SPEED_VAR) is still set to 12.31, the position varaince (POSITION_VAR) is set to be 4, the orientaion angle varaince (ANGLE_VAR) is set to be ${\dfrac{\pi^2}{100}}$.

<div align=center>
<img src="./figs/speedvariance.png" style="width:30%" />
<figcaption>Speed variance measurement</figcaption>
</div>

The final parameter settings are shown as below.

$$ 
\mathbf{R} = diag([cst.RPX, cst.RPY, cst.RAN, cst.RRL, cst.RRW])
$$
$$ 
\mathbf{R_{encoder}}
=
\begin{bmatrix}
0 & 0 \\
0 & 0 \\
0 & 0 \\
cst.RLW & 0 \\
0 & cst.RRW
\end{bmatrix}
$$
Here, for the readability of the code, we have put all constants in to a file call constants_robot.py and import it here as cst. For the observation noise matrix R, we just set RPX=RPY=POSITION_VAR, RAN=ANGLE_VAR, RLW=RRW=SPEED_VAR/2 as we obtain the variance parameters through observation.

$$ 
\mathbf{Q}= diag([cst.QPX, cst.QPY, cst.QAN, cst.QRL, cst.QRW])

$$
For process noise matrix, according to the lecture in Multivariable Control course, the process noise covariances are better to set to large values so the kalman filter can be more robust against the possibly non-whiteness of process noise. This method does not always works but is reasonable so we decide to give it a try. We set QPX=QPY=4*POSITION_VAR, QAN=4*ANGLE_VAR, QLW=QRW=4*SPEED_VAR. To be noticed, in kalman filter, the speed measurements from wheel encoders cannot be directly used as they are not the true speed of thymio. To get the true speed, we have to multiply the measurements with speed conversion factor which is about 0.435 obtained in the exercise by using black and white strips.

In [ ]:
# code implemetation in kalman.py file
def kalman_func(kalman: ExtendedKalmanFilter(position=np.array([0, 0, 0])) = ExtendedKalmanFilter, position = None, wheelspeed = None):

    kalman.get_dt()
    kalman.set_state_transition_matrix()
    kalman.predict()
    kidnapping = False
    
    # check if the robot has vision
    if position is not None:
        measurement = np.array([position[0], position[1], position[2], wheelspeed[0], wheelspeed[1]])
        kalman_est_pos, kalman_variance = kalman.update_vision(measurement)
        distance = np.linalg.norm(kalman_est_pos[0:2] - position[0:2], 2)
        angle_diff = convert_angle(kalman_est_pos[2] - position[2])
        if distance > cst.DIST_TRESHOLD or abs(angle_diff) > cst.ANGLE_TRESHOLD:
            print("Detecting kidnapping")
            kalman.get_dt()
            kidnapping = True
            kalman.init_state_vector(position, wheelspeed)
    else:
        kalman_est_pos, kalman_variance = kalman.update_encoder(wheelspeed)
        print("No vision, only encoder")

    return kalman_est_pos, kalman_variance, kidnapping   


#### Implementation
As shown in the code above, we have created a EKF class so that the code can be clean and readable. The input of the kalman_func() funciton is the EKF class, the position measurement, the wheel speed meaasured and the sampling time for simluation and the output of the funciton is the estimated positiona and boolean variable telling us whether the thymio is kidnapped or not.

Fistly, we check the validation of the position measured. If the postion measured fails (is None), then we update the position estimation simply with EKF dynamics. Otherwise, we update the position with both webcam measaured postion and self predicted positions. Then we calculate the sampling period dt and update the time stamp stored in the EKF class. Finally, we updated the postion with estimated one and do the prediction by calling the EKF class functions which do the predict and update steps as mentioned in EKF instruction.

For the decision of whether the thymio is kidnapped, we compute the difference between the webcam meansure postion and the estimated postion. If the difference is large enough ( greater than cst.DIST_TRESHOLD and cst.ANGLE_TRESHOLD), then we decide that the thymio robot is kidnapped and we reinitailized the EKF with the webcam measured postion as it is more reliable compared with wheel speed encoders which cannot sense the exterior forces.


#### Simulation

We have done a simlution on the computer show the effect of the kalman filter. The trajectory of the point is also controlled by the right and left speed. Then we have manually add noises on the postion measured and wheel speed measured respectively. The outcome of kalman filter is shown as below. It can be seen that, the kalan filter really corrects the postion by fusing the webcam measurements and the wheel speed encoder measurements. The code for simulation can be found in vis_kal.ipynb.


<div align=center>
<img src="./figs/kalman_sim.png" style="width:30%" />
<figcaption>Kalman filter simulation</figcaption>
</div>

# Conclusion